## 准备数据

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [2]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]


## 建立模型

In [3]:
class myModel:
    def __init__(self):
        self.W1 = tf.Variable(shape=[28*28, 100], dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[28*28, 100], minval=-0.1, maxval=0.1))
        self.W2 = tf.Variable(shape=[100, 10], dtype=tf.float32,
                              initial_value=tf.random.uniform(shape=[100, 10], minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[100], dtype=tf.float32, initial_value=tf.zeros(100))
        self.b2 = tf.Variable(shape=[10], dtype=tf.float32, initial_value=tf.zeros(10))
    def __call__(self, x):
        flat_x = tf.reshape(x, shape=[-1, 28*28])
        h1 = tf.tanh(tf.matmul(flat_x, self.W1) + self.b1)
        logits = tf.matmul(h1, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [6]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)

    optimizer.apply_gradients(zip(grads, trainable_vars))

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [7]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.052431 ; accuracy 0.46143332
epoch 1 : loss 1.9716214 ; accuracy 0.5181
epoch 2 : loss 1.8937765 ; accuracy 0.56378335
epoch 3 : loss 1.8187805 ; accuracy 0.60295
epoch 4 : loss 1.7465534 ; accuracy 0.63566667
epoch 5 : loss 1.6770686 ; accuracy 0.66216666
epoch 6 : loss 1.6103569 ; accuracy 0.6844
epoch 7 : loss 1.5464838 ; accuracy 0.7024
epoch 8 : loss 1.4855217 ; accuracy 0.7163333
epoch 9 : loss 1.427523 ; accuracy 0.72755
epoch 10 : loss 1.3725008 ; accuracy 0.73686665
epoch 11 : loss 1.3204236 ; accuracy 0.74545
epoch 12 : loss 1.2712207 ; accuracy 0.754
epoch 13 : loss 1.2247914 ; accuracy 0.76245
epoch 14 : loss 1.1810154 ; accuracy 0.7704
epoch 15 : loss 1.1397597 ; accuracy 0.7781
epoch 16 : loss 1.100885 ; accuracy 0.7855333
epoch 17 : loss 1.0642495 ; accuracy 0.7921
epoch 18 : loss 1.0297107 ; accuracy 0.79803336
epoch 19 : loss 0.9971287 ; accuracy 0.80373335
epoch 20 : loss 0.9663677 ; accuracy 0.8088
epoch 21 : loss 0.93729824 ; accuracy 0.8138
epoch 2